In [2]:
## Preamble: Package Loading
import numpy as np
from sklearn import linear_model
from IPython.display import display
import matplotlib.pyplot as plt
from matplotlib import gridspec
import pandas as pd
import itertools as iter
import os
import datetime as dt
import json
# Preamble working directory retreival
wkng_folder = os.getcwd()

In [3]:
input_filename = 'pscdata_6_19_1923.json'
data_file = '/Users/ericpenner/Google_Drive/Research/pan_sel_cntrl/data'
input_file_full = ''.join([data_file,'/',input_filename])
with open(input_file_full) as f_obj: 
    pscdata = json.load(f_obj)
# Initializing the data sets metadata dictionary 
inpt = pscdata[0][0].copy()

k=10
data_err = pd.DataFrame(pscdata[k][0]['err_df'], columns = pscdata[0][1]['Derr_nms'])  
data_inst = pd.DataFrame(pscdata[k][0]['inst_df'], columns = pscdata[0][1]['Dins_nms'])
data_pan = pd.DataFrame(pscdata[k][0]['prim_df'], columns = pscdata[0][1]['Dlng_nms'])

In [4]:
inpt

{'c_inst': 3,
 'cin': 'crs',
 'dep_nm': 'Y',
 'en_nm': ['Z1,1', 'Z1,2'],
 'err_vpro': [0.8, 0.36],
 'ex_nm': ['Z2,1', 'Z2,2'],
 'ex_vpro': [0.5],
 'frc': 0,
 'inst_vpro': [0.5, 0.25, -0.1],
 'n_end': 2,
 'n_exo': 2,
 'ncs': 10,
 'nds': 1000,
 'ntp': 30,
 'r_seed': 19977,
 'sec_pan': 1,
 't_inst': 10,
 'tin': 't'}

In [5]:
pd.DataFrame(pscdata[0][1]['coeff'][1])

,0,1,2,3,4,5,6,7,8,9,10,11
0,1.0,-1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,-1.0,0.0,1.0
1,1.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,1.0,1.0
2,1.0,-1.0,0.0,0.0,0.0,0.0,-1.0,-1.0,-1.0,0.0,0.0,0.0
3,1.0,-1.0,-1.0,0.0,0.0,0.0,0.0,-1.0,0.0,-1.0,0.0,0.0
4,1.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,1.0,1.0
5,1.0,-1.0,0.0,0.0,0.0,0.0,0.0,-1.0,-1.0,-1.0,0.0,0.0
6,1.0,-1.0,0.0,0.0,0.0,0.0,0.0,-1.0,-1.0,0.0,0.0,1.0
7,1.0,-1.0,0.0,0.0,-1.0,1.0,0.0,-1.0,0.0,0.0,0.0,0.0
8,1.0,-1.0,0.0,1.0,0.0,0.0,0.0,-1.0,0.0,-1.0,0.0,0.0
9,1.0,-1.0,0.0,0.0,-1.0,0.0,0.0,0.0,-1.0,-1.0,0.0,0.0


<h2>Estimator With Only ScikitLearn Lasso</h2>

In [6]:
dp = data_pan
di = data_inst
de = data_err

dep_nm = inpt['en_nm']
ex_nm = inpt['ex_nm']
n_exo = len(ex_nm)
ins_nm = list(di.columns)[1:]
adep_nm = [ 0 for i in range(len(dep_nm))]
epsil = 0.05
ain_cf = [ 0 for i in range(len(dep_nm))]
ain_cf_rm = [ 0 for i in range(len(dep_nm))]
excf1 = []

for g in range(len(en_nm)):
    excf1 = []
    for k in range(inpt['ncs']):
        dsl = dp.loc[dp['crs']==k+1,['t'] + [dep_nm[g]] + ex_nm]
        dsl = pd.merge(dsl,di,how ="inner", on='t')
        lin_reg = linear_model.LinearRegression()
        lin_reg.fit(dsl.loc[:,ex_nm+ins_nm].values,dsl.loc[:,dep_nm[g]].values.reshape(dsl.shape[0],1))
        excf1.append([lin_reg.coef_[0][i] for i in range(inpt['n_exo'])])

    incf = [0 for i in range(inpt['t_inst'])]    
    excf = [np.mean([excf1[i][j] for i in range(len(excf1))] ) for j in range(n_exo)]

    adep_nm[g] = ''.join(['a',dep_nm[g]])
    dp[adep_nm[g]] = (dp.loc[:,dep_nm[g]].values.reshape(dp.shape[0],1)
                     -dp.loc[:,ex_nm].values.dot(np.array(excf).reshape(inpt['n_exo'],1)))
    
    ain_cf1= []
    for k in range(inpt['ncs']):
        ds2 = dp.loc[dp['crs']==k+1,['t'] + [adep_nm[g]] ]
        ds2 = pd.merge(ds2,data_inst,how ="inner", on='t')
        lasso_reg = linear_model.Lasso(alpha = 0.4)
        lasso_reg.fit(ds2.loc[:,ins_nm].values,ds2.loc[:,adep_nm[g]].values.reshape(ds2.shape[0],1))
        ain_cf1.append(list(lasso_reg.coef_))

    ain_cf[g] = excf
    for j in  range(len(ain_cf1[0])): 
        a = [ain_cf1[i][j] for i in range(len(ain_cf1)) if np.abs(ain_cf1[i][j]) > epsil]
        if not a:
            ain_cf[g].append(0)
        else:
            ain_cf[g].append(np.mean(a))    

    ain_cf_rm[g] = [[1,1] + [int(np.abs(ain_cf1[j][i])>epsil) 
                            for i in range(len(ain_cf1[0]))] 
                            for j in range(len(ain_cf1))]

In [7]:
ain_cf

[[-0.9320210439493026,
  0.997178480043544,
  0.04627244632487739,
  -1.0393972950714356,
  0.5167090560403694,
  0.8742605310534153,
  0.3236071850068241,
  0.4274485594704796,
  -0.07472347053819012,
  0.46660589161111454,
  -1.0913422830285149,
  -0.5285170333898953],
 [1.0589440008103959,
  -1.0163103665460518,
  0,
  0,
  -0.2974856764910138,
  0.44936522192650274,
  -0.211256980292427,
  -0.6041313838006263,
  -0.4043201902325079,
  -0.488780759273093,
  0.2609506495311962,
  0.6968315607916983]]

In [8]:
pd.DataFrame(ain_cf1)

,0,1,2,3,4,5,6,7,8,9
0,-0.00000,0.0000,0.000000,0.449365,0.208153,-0.000000,-0.000000,-0.000000,0.000000,0.306967
1,-0.00000,0.0000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,0.532747,0.755600
2,0.00000,0.0000,-0.000000,-0.000000,-0.630667,-1.162640,-0.212077,-0.000000,-0.000000,0.000000
3,-0.00000,-0.0000,-0.000000,-0.000000,0.000000,-0.209925,-0.000000,-0.376849,-0.000000,0.000000
4,0.00000,0.0000,0.000000,0.000000,-0.000000,-0.081423,-0.179932,0.000000,0.316568,1.225309
5,-0.00000,-0.0000,0.000000,-0.000000,-0.000000,-0.687390,-0.824936,-0.915903,-0.000000,0.000000
6,0.00000,0.0000,0.000000,-0.000000,-0.000000,-1.077360,-0.096513,-0.000000,-0.000000,0.499450
7,-0.00000,-0.0000,-0.131372,0.000000,-0.000000,-0.062002,-0.000000,0.000000,0.000000,0.000000
8,0.00000,0.0106,0.000000,-0.000000,-0.000000,-0.948180,-0.107136,-0.173590,-0.000000,-0.000000
9,0.00551,0.0000,-0.463600,-0.024132,-0.000000,-0.000000,-1.005327,-0.000000,-0.066463,-0.000000


In [9]:
excf1

[[1.0745527437101308, -1.1940733380049862],
 [0.8966480758031982, -0.6990502598348142],
 [0.6463010838920157, -0.39919114519071924],
 [0.9165648827011966, -0.6119711550621931],
 [1.2809464651645122, -1.133297411823245],
 [1.4729194101312866, -1.5620606858156143],
 [1.054625938414029, -1.07244022220629],
 [0.9965102968080453, -1.1235018249926136],
 [1.1092580764938462, -1.137956777888309],
 [1.1411130349856966, -1.2295608446417328]]

<h2> Making it a function </h2>

<h3> Necessary OLS function </h3>

In [24]:
def ols(df,inpt):
    """
INPUTS
df                (pandas df) Data Frame with all regressors
inpt              (dict) Dictionary with the following
  inpt['dep']     (string) Name of dependent variable contained in df
  inpt['reg']     (list of strings) names of regressors in df 
  inpt['cons']    (0,1) Indicator for whether a constant should be included

OUTPUTS 
out               (list of lists) List of the following
  out[0]          (list) Estimated coefficients
  out[1]          (list) Residuals
  out[2]          (list) Estimated conditional expectation
    """
    # Extracting input variables
    dep = inpt['dep']
    reg = inpt['reg']
    cons = inpt['cons']
    # Determining length of df (number of obs)
    n = df.shape[0]
    # Extracting Dependent Variable from df
    Y = df.loc[:,dep].values.reshape(n,1)
    # Extracting Regressors from df
    if len(reg) == 1:
        X = df.loc[:,reg].values.reshape(n,1)
    elif len(reg) > 1: 
        X = df.loc[:,reg].values
    # Adding column of ones if a constant is included
    if cons == 1: 
        X = np.hstack((np.ones((n,1)),X))
    # Estimated regression coefficients
    alpha = np.linalg.inv(X.T.dot(X)).dot(X.T.dot(Y))
    # Estimated Conditional Expectation
    Yhat = X.dot(alpha)
    # Residuals of the regression
    res = Y - X.dot(alpha)
    # Constructing output list of lists
    out = [alpha.T.tolist()[0],res.T.tolist()[0],Yhat.T.tolist()[0]]
    return out

In [25]:
dp_in = data_pan
di_in = data_inst
inpt = pscdata[0][0].copy()

# Indicator for  whether in this run the subset of instrument relvant to each crs is known.
inpt['kwnsub'] = 0 

# Indicator for whether residuals are observed
inpt['orcl'] = 0
inpt['lasso'] = 1
inpt['dep'] = inpt['en_nm'][0]
inpt['reg'] = inpt['ex_nm']
# List of list with the names of the relevant instruments for each crossection
in_nm=[]
for i in range(inpt['ncs']):
    # If the subset is known then list of relevant inst. for each crs is supplied to estimator
    if inpt['kwnsub'] == 1:
        a=[ True if pscdata[0][1]['coeff'][0][i][k]!=0 else False 
            for k in range(inpt['n_exo'],inpt['n_exo']+inpt['t_inst'])]
        in_nm.append(np.array(pscdata[0][1]['Dins_nms'][1:])[a].tolist())
    # If the subset is unknown then list of all inst. will be supplied to est. for each crs
    else:
        in_nm.append(pscdata[0][1]['Dins_nms'][1:])
inpt['in_nm'] = in_nm
inpt

{'c_inst': 3,
 'cin': 'crs',
 'dep': 'Z1,1',
 'dep_nm': 'Y',
 'en_nm': ['Z1,1', 'Z1,2'],
 'err_vpro': [0.8, 0.36],
 'ex_nm': ['Z2,1', 'Z2,2'],
 'ex_vpro': [0.5],
 'frc': 0,
 'in_nm': [['W1', 'W2', 'W3', 'W4', 'W5', 'W6', 'W7', 'W8', 'W9', 'W10'],
  ['W1', 'W2', 'W3', 'W4', 'W5', 'W6', 'W7', 'W8', 'W9', 'W10'],
  ['W1', 'W2', 'W3', 'W4', 'W5', 'W6', 'W7', 'W8', 'W9', 'W10'],
  ['W1', 'W2', 'W3', 'W4', 'W5', 'W6', 'W7', 'W8', 'W9', 'W10'],
  ['W1', 'W2', 'W3', 'W4', 'W5', 'W6', 'W7', 'W8', 'W9', 'W10'],
  ['W1', 'W2', 'W3', 'W4', 'W5', 'W6', 'W7', 'W8', 'W9', 'W10'],
  ['W1', 'W2', 'W3', 'W4', 'W5', 'W6', 'W7', 'W8', 'W9', 'W10'],
  ['W1', 'W2', 'W3', 'W4', 'W5', 'W6', 'W7', 'W8', 'W9', 'W10'],
  ['W1', 'W2', 'W3', 'W4', 'W5', 'W6', 'W7', 'W8', 'W9', 'W10'],
  ['W1', 'W2', 'W3', 'W4', 'W5', 'W6', 'W7', 'W8', 'W9', 'W10']],
 'inst_vpro': [0.5, 0.25, -0.1],
 'kwnsub': 0,
 'lasso': 1,
 'n_end': 2,
 'n_exo': 2,
 'ncs': 10,
 'nds': 1000,
 'ntp': 30,
 'orcl': 0,
 'r_seed': 19977,
 'reg': ['Z2,

In [26]:
# def panel_fe(dp_in,di_in,inpt):
#     """
# INPUTS
# dp_in                     (pandas df) df with dependent var. and all exogenous regs
# di_in                     (pandas df) df with all instruments
# inpt                      (dict) Dictionary with the following
#   inpt['dep']               (string) Name of dependent variable contained in dp
#   inpt['reg']               (list of strings) names of exogenous regressors
#   inpt['cin']               (string) name of crossection index in dp
#   inpt['tin']               (string) name of time index in dp
#   inpt['ncs']               (int) number of crossections
#   inpt['in_nm']             (list of lists of strings) with the following components
#     input['in_nm'][i-1]       (list of names) names of instruments relevant to crs i 
  

# OUTPUTS 
# out               (list of lists) List of the following
#   out[0]          (list) Estimated coefficients
#   out[1]          (list) list of all relevant instrument
#   out[2]          (list) Estimated error terms Vi_j
#     """

## Extracting Variables from inpt dictionary
dep = inpt['dep']
reg = inpt['reg']
in_nm = inpt['in_nm']
cin = inpt['cin']
tin = inpt['tin']
ncs = inpt['ncs']
lasso = inpt['lasso']
di  = di_in
dp  = dp_in

## Constructing a df of all instrument relevant to at least 1 crossection
# Initializing the set of all included instruments as the 1st set relevant inst
inc = set(in_nm[0])
# Collecting rest of relevant instruments
for i in range(len(in_nm)):
    # Union of inc and ith set of relevant instruments
    inc = inc|set(in_nm[i])
# All included Instruments listed in order index order
inc = [''.join(['W',str(i)]) for i in range(1,di.shape[1]) if ''.join(['W',str(i)]) in inc]
# df with time index and all included instruments
di_inc = di.loc[:,[tin] + inc]

# List of logical vectors (as list) of which instruments in inc are relevant to ith crs
in_vec = [[ 1 if inc[i] in in_nm[j] else 0 for i in range(len(inc))]
              for j in range(ncs)]

# First differenced included instrument df
Di = di_inc.loc[:,[tin]]
for j in range(0,len(inc)):
    # Adding D to name of jth instrument in inc
    D_nm = ''.join(['D', di_inc.columns[j+1]])
    # First difference of jth instrument in inc
    Di[D_nm] = (di_inc.loc[:,di_inc.columns[j+1]].values 
                - di_inc.loc[:,di_inc.columns[j+1]].shift(1).values)

## Constructing Panel Version of relevant instrument data
for i in range(ncs):
    # Initializing differenced panel template df
    a1 = Di.loc[:,tin].copy()
    # Adding the crossection variable
    a1 = pd.concat([a1,pd.DataFrame(np.ones((di_inc.shape[0],1))*(i+1)
                                    ,columns = [cin])],axis = 1)
    # Initializing the panel template df
    b1 = a1
    # Product of Di with in_vec[i-1] s.t. inst. not relevant to i are zero 
    a2 = pd.DataFrame(Di.iloc[:,1:].values.dot(np.diag(in_vec[i]))
                      ,columns = Di.columns[1:])
    # Product of di_inc with in_vec[i-1] s.t. inst. not relevant to i are zero
    b2 = pd.DataFrame(di_inc.iloc[:,1:].values.dot(np.diag(in_vec[i]))
                      ,columns = di_inc.columns[1:])
    # Concatenating a2 onto panel template adding [cin] and [tin]
    a2 = pd.concat([a1,a2],axis = 1)
    # Concatenating b2 onto panel template adding [cin] and [tin]
    b2 = pd.concat([b1,b2],axis = 1)
    if i == 0:
        # if  i = 0 initialize final differenced panel df
        Ddi_pan = a2.iloc[1:,:]
        # if i = 0 initialize final panel df
        di_pan = b2
    elif i > 0: 
        # if i > 0 add ith crosssection rows onto final
        Ddi_pan = pd.concat([Ddi_pan,a2.iloc[1:,:]], axis = 0)
        # if i > 0 add ith crosssection rows onto final
        di_pan =  pd.concat([di_pan,b2], axis = 0)

## First Difference dependent and exogenous regressor matrix..
for i in range(1,ncs+1):
    # Initializing panle template df
    c1 = dp.loc[dp[cin]== i,[cin]+[tin]].copy() 
    # First differencing al relevant variables for i crs
    for j in [inpt['dep']] + inpt['reg']:
        c1[''.join(['D',j])] = (dp.loc[dp[cin]== i,j].values 
                    - dp.loc[dp[cin]== i,j].shift(1).values)
    if i == 1:
        # If i = 1 initialize final panel df
        c2 = c1.iloc[1:,:]  
    elif i > 1:
        # If i > 1 add onto final panel df
        c2 = pd.concat([c2,c1.iloc[1:,:]],axis = 0)

## OLS estimation
# Merging all differenced panel df's together
Ddi_pan = pd.merge(c2,Ddi_pan,on=[cin,tin],how = 'inner')
# List of all regressor names in Ddi_pan for use in ols()
Dregs = ([''.join(['D',reg[i]]) for i in range(len(reg))] 
        + [''.join(['D',inc[i]]) for i in range(len(inc))])
# Dictionary for input into ols()
inpt_ols = {'dep': ''.join(['D',dep]) , 'reg': Dregs, 'cons': 0}
# Estimation by ols()
ols_out = ols(Ddi_pan,inpt_ols)


# Names of all exogenous regressor and inst. included in di_pan
regs = ([ reg[i] for i in range(len(reg))] 
        + [ inc[i] for i in range(len(inc))])
# Estimated coefficient from estimator above 
e_cf = np.array(ols_out[0]).reshape(len(ols_out[0]),1)

# Constucting a panel df of esitmated errors Vj,i
for i in range(1,ncs+1):
    # np.array of dep variable values for ith crs
    d1 = dp.loc[dp[cin] == i,[dep]].values
    # df of exogenous regressor values for ith crs
    d21 = dp.loc[dp[cin] == i ,[cin]+[tin]+reg]
    # included instruments panel with indexes for i 
    d22 = di_pan.loc[di_pan[cin] == i,[cin]+[tin]+inc]
    # merging d21 and d22 making a df with all RHS regressors 
    d23 = pd.merge(d21,d22,on = [cin,tin], how = 'inner')
    # Estimated time varying component of dependent variable
    d2 = d23.drop([cin,tin],axis = 1).values.dot(e_cf)
    # Residual of time varying component 
    d3 = d1.reshape(d1.shape[0],1) - d2
    # Centered residula of time varying component
    d3 = d3 - np.mean(d3)
    if i == 1:
        # if i = 1 initialize panel df
        p_res = [list(d3.T[0])]
    elif i > 1:
        # if i > 1 add onto p_res
        p_res.append(list(d3.T[0]))

# Output of the function
out = [list(e_cf.T[0]), inc , p_res ]
#return out